In [ ]:
# The aim of this module is to company value and expected growth rate using Ben Graham's formula

# File name convention
# R_ = raw extract 
# E_ = Entry files into enrichment layer
# C_ = enriched layer with calculation
# I = insights layer, designed for model baselines
# V_ = validation

In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

# I_FLAT = pd.read_pickle ('/Users/joezhou/Downloads/C_TICKER_FLAT.pkl') 
# C_BG_MODEL = pd.read_pickle ('/Users/joezhou/Downloads/C_BEN_G_MODEL.pkl') 
I_FLAT = pd.read_pickle ('/Users/joezhou/Downloads/Investment/1 data/C_TICKER_FLAT.pkl') 


In [2]:
# from datetime import datetime as dt
# dte = dt.now().strftime('%Y%m%d')
# fname = "/Users/joezhou/Downloads/Investment/TEST_I_FLAT{}.xlsx".format(dte)
# I_FLAT.to_excel(fname)

In [2]:
# valuation using NTA approach
DF_VAL1 = I_FLAT 
DF_VAL1['CALC_TA_PS'] = DF_VAL1['Net Tangible Assets']/DF_VAL1['sharesOutstanding']
DF_VAL1['CALC_VAL_NTA_DIV'] = DF_VAL1['CALC_TA_PS'] + (DF_VAL1['CALC_DIV_10YR_AMT']*.8)

DF_VAL1 = DF_VAL1[['TickName','CALC_VAL_NTA_DIV','CALC_TA_PS','CALC_DIV_10YR_AMT']]

# Note: need to discount cashflows & find median NTA over the last few years


In [10]:
# Valuation using dividends
# https://python.plainenglish.io/dividends-discount-model-in-python-13c0f141a611

import yfinance as yf # this is to get the stock data
import pandas as pd # this is for working on data frames
import numpy as np # this is to manipulate the data

# 1. download data
ticker = yf.Ticker("CBA.AX") 
stock = ticker.actions

# 2. adjust for splits
stock_split = stock["Stock Splits"].to_numpy()
stock_split_replaced = np.where(stock_split == 0, 1, stock_split)
stock_split_comp = np.cumprod(stock_split_replaced, axis=0)
stock["stocksplit_adj"] = stock_split_comp.tolist()
stock["div_adj"] = stock["Dividends"] * stock["stocksplit_adj"]

# find yearly dividend and median growth
stock['year'] = stock.index.year
stock_grp = stock.groupby(by=["year"]).sum()
stock_grp["div_PCT_Change"] = stock_grp["div_adj"].pct_change(fill_method ='ffill')
median_growth = stock_grp["div_PCT_Change"].median()
# find the last dividend
lst_Div = stock_grp.at[2021,'Dividends']
# find information to calculate price 
beta = ticker.info["beta"]
risk_free_rate = 0.03
mkt_return = .09
MKT_Risk_prem = mkt_return - risk_free_rate
COE = round(beta * MKT_Risk_prem + risk_free_rate,4)
# find faire price
# fair_sharePrice = round(exp_future_div/(COE-median_growth),2)
fair_sharePrice = round(lst_Div/(COE-median_growth),2)


In [ ]:
# Calculate Ben Graham intrensic valuations and revse engineer a target growth rate based on latest closing price

# the current yield on AAA corporate bonds (CURR_AAA_Y)
CURR_AAA_Y = 3.38
# the average yield of AAA corporate bonds (AVG_BND_Y)
AVG_BND_Y = 4.4
# EPS of non growing company (EPS_NG)
EPS_NG = 8.5

# reasonably expected 7 to 10 Year Growth Rate of EPS (EPS_G = 8.5)
EPS_G = ADDED_PRICE_INF['earningsQuarterlyGrowth']*3

# actual formula
# ADDED_PRICE_INF['CALC_BF_IV'] = ADDED_PRICE_INF['trailingEps']*(EPS_NG+2*EPS_G))*AVG_BND_Y/CURR_AAA_Y
# conservative
ADDED_PRICE_INF['CALC_BF_IV'] = ADDED_PRICE_INF['trailingEps']*(EPS_NG+EPS_G)*AVG_BND_Y/CURR_AAA_Y

ADDED_PRICE_INF['CALC_BF_GRWTH'] = (ADDED_PRICE_INF['Last_Close_price']*CURR_AAA_Y/AVG_BND_Y/ADDED_PRICE_INF['trailingEps']-EPS_NG)/2



In [11]:
# BG_MODEL.to_pickle("/Users/joezhou/Downloads/C_CALCS.pkl")
# BG_MODEL.to_excel('/Users/joezhou/Downloads/C_CALCS.xlsx', engine='xlsxwriter')  

DF_VAL1.to_pickle("/Users/joezhou/Downloads/Investment/1 data/C_VALUATION.pkl")
